In [1]:
import math
import pandas as pd
import numpy as np
import itertools
from itertools import tee, repeat
from progressbar import ProgressBar
import os, datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, metrics
import math


In [2]:
os.chdir('../')


In [3]:
# data_train['event time:timestamp'] = pd.to_datetime(data_train['event time:timestamp'])
data_train = pd.read_csv('./data/2017-train.csv', error_bad_lines=False)
data_train['event time:timestamp'] = pd.to_datetime(data_train['event time:timestamp'])

data_test = pd.read_csv('./data/2017-test.csv', error_bad_lines=False)
data_test['event time:timestamp'] = pd.to_datetime(data_test['event time:timestamp'])

df2_train = data_train
df2_test = data_test

# Convert from string to datetime
df2_train['event time:timestamp'] = pd.to_datetime(df2_train['event time:timestamp'])
df2_test['event time:timestamp'] = pd.to_datetime(df2_test['event time:timestamp'])

# Concatenate the datasets
df2 = pd.concat([df2_train, df2_test])

# Sort the values and reset the index accordingly
df2 = df2.sort_values(by=['event time:timestamp'])
df2 = df2.reset_index(drop=True)

# Create dataframe with first events of each case in chronological order
df_cases_first_event = df2.drop_duplicates(subset='case concept:name', keep='first')
df_cases_first_event = df_cases_first_event.reset_index(drop=True)

# Split the cases in 80-20
index80 = int(len(df_cases_first_event) * 80 / 100 - 1)
df_cases_train = df_cases_first_event.loc[:index80]
df_cases_test = df_cases_first_event.loc[index80 + 1:]

# Training and test set complete
df_train_new = df2.loc[df2['case concept:name'].isin(list(df_cases_train['case concept:name']))]
df_test_new = df2.loc[df2['case concept:name'].isin(list(df_cases_test['case concept:name']))]

# Setting the 'now' (as the last event of the last case in the training set)
# now = df_train_new[df_train_new['case concept:name']==df_cases_train.iloc[-1]\
#    ['case concept:name']].iloc[-1]['event time:timestamp']
if 'case LoanGoal' in df2_train:
    now = pd.Timestamp('2016-12-08')
else:
    now = pd.Timestamp('2012-12-12')

# Training and test set before 'now'
df_train_new_now = df_train_new.loc[(df_train_new['event time:timestamp'] <= now)]
df_test_new_now = df_test_new.loc[(df_test_new['event time:timestamp'] <= now)]

data_train = df_train_new_now
data_test = df_test_new_now

In [4]:
data_train = data_train.sort_values(by=['case concept:name', 'event time:timestamp'])
data_test = data_test.sort_values(by=['case concept:name', 'event time:timestamp'])

# ADD
if 'case LoanGoal' in data_train.columns:
    data_train['case LoanGoal'] = data_train['case LoanGoal'].apply(
        lambda x: 'Other - see explanation' if x == 'Other, see explanation' else x)
    
    data_test['case LoanGoal'] = data_test['case LoanGoal'].apply(
        lambda x: 'Other - see explanation' if x == 'Other, see explanation' else x)
    datetimeFormat = '%Y-%m-%d %H:%M:%S.%f'

    pt1, pt2, pt3 = 4, 8, 12
else:  # check if raod data
    datetimeFormat = '%Y-%m-%d'

    pt1, pt2, pt3 = 2, 3, 5



data_train.to_csv("./fixed_train.csv")

log = dict()
# print('making dictionary from fixed file')
with open('./fixed_train.csv', 'r') as f:
    next(f)

    for line in f:
        line = line.strip()
        if len(line) == 0:
            continue
        parts = line.split(',')
        caseid = parts[pt1]
        task = parts[pt2]
        timestamp = parts[pt3]

        if caseid not in log:
            log[caseid] = [[], []]

        log[caseid][0].append(task)
        log[caseid][1].append(timestamp)

f.close()

os.remove('./fixed_train.csv')

for i in log.keys():  # updating the dictionary to contain also all next events
    current = log[i][0]  # recording the cuurent case' events

    real_next = current[1:]  # next real events
    real_next.append('New Case')  # adding a 'new case' as real next event for every last event

    log[i].append(real_next)  # adding the real next events to the log file

#  Repeating the same process from above on the test data.

data_test.to_csv("./fixed_test.csv")

log_test = dict()
# print('making test dictionary from fixed file')
with open('./fixed_test.csv', 'r') as f:
    next(f)

    for line in f:
        line = line.strip()
        if len(line) == 0:
            continue
        parts = line.split(',')
        caseid = parts[pt1]
        task = parts[pt2]
        timestamp = parts[pt3]

        if caseid not in log_test:
            log_test[caseid] = [[], []]

        log_test[caseid][0].append(task)
        log_test[caseid][1].append(timestamp)

f.close()

os.remove('./fixed_test.csv')

"""Fixing a bug of cases that are in the test data but are incomplete due to the train-test split."""

bugs = []

for i in log_test.keys():  # recording the cases which have events cut because of the train - test split
    if len(log_test[i][0]) == 1:
        bugs.append(i)

for x in bugs:  # deleting the above mentioned events
    del log_test[x]
    data_test.drop(data_test.index[data_test['case concept:name'] == x], inplace=True)

for i in log_test.keys():
    current = log_test[i][0]  # current case' events

    real_next = current[1:]  # next real events
    real_next.append('New Case')  # adding a 'new case' as real next event for every last event
    log_test[i].append(real_next)

m = 0

for i in log.keys():
    if len(log[i][0]) > m:
        m = len(log[i][0])


delete = []
for i in log_test.keys():
    if len(log_test[i][0]) > m:
        delete.append(i)

for i in delete:
    data_test.drop(data_test.index[data_test['case concept:name'] == i], inplace=True)

    del log_test[i]

In [5]:
#  new dictionary that will contain for every position(key) the observed traces and next events for each trace(values)
#  so case [A, B, C] would be saved as {0:[[A],[B]], 1: [[A,B], [C]], 2: [[A, B, C], [New Case]]}
train_data = {}

for i in log.keys():
    count = 0
    for x in log[i][0]:
        case = log[i][0]
        # ind = log[i][0].index(x)

        if count not in train_data:  # making the two lists in the dictionary
            train_data[count] = [[],
                                 []]  # list 1 is all for all traces of the position, list 2 is for all next events

        train_data[count][0].append(case[:count + 1])  # appending the trace

        if count < len(case) - 1:
            train_data[count][1].append(case[count + 1])  # appending the next event of the trace

        elif count == len(case) - 1:
            train_data[count][1].append('New Case')

        count += 1

In [6]:
#  repeating the same process on the test data
test_data = {}

for i in log_test.keys():
    count = 0
    for x in log_test[i][0]:
        case = log_test[i][0]
        # ind = log_test[i][0].index(x)

        if count not in test_data:
            test_data[count] = [[], []]

        test_data[count][0].append(case[:count + 1])  # appending the trace

        if count < len(case) - 1:
            test_data[count][1].append(case[count + 1])  # appending the next event of the trace

        elif count == len(case) - 1:
            test_data[count][1].append('New Case')

        count += 1

In [7]:
# encoding all unique event names of all the data into integers

cases = list(data_train['event concept:name'].unique()) + list(
    data_test['event concept:name'].unique())  # all events
cases.append('New Case')  # adding the 'New Case' because we predict next event is going to be new case
cases = list(set(cases))
le = preprocessing.LabelEncoder()
le.fit(cases)  # encoding all event names into integers

pbar = ProgressBar()
print('[10/13]')

for i in pbar(train_data.keys()):  # the dictionaries from above are encoded into integers

    encoded = []
    for trace in train_data[i][0]:  # encoding all strings of a trace, can be multiple if case lenght is more than 2
        local_encoded = []
        for event in trace:
            local_encoded.append(int(le.transform([event])))  # transforming into integer
        encoded.append(local_encoded)

    train_data[i][0] = np.array(encoded)  # making the list with integers into array so the tree can take it

    encoded_next = []  # encoding all strings of next events for a trace, its always length 1 !
    for g in train_data[i][1]:
        encoded_next.append(int(le.transform([g])))  # transforming into integer

    train_data[i][1] = np.array(encoded_next)  # making the list with integers into array

[10/13]


100% |########################################################################|


In [8]:
# repeating the procedure from above on the test data

pbar = ProgressBar()
print('[11/13]')

for i in pbar(test_data.keys()):

    encoded = []
    for trace in test_data[i][0]:
        local_encoded = []
        for event in trace:
            local_encoded.append(int(le.transform([event])))
        encoded.append(local_encoded)

    test_data[i][0] = np.array(encoded)

    encoded_next = []
    for g in test_data[i][1]:
        encoded_next.append(int(le.transform([g])))

    test_data[i][1] = np.array(encoded_next)

# Function for training decision tree for any given position (as long as the position is in the train data)

[11/13]


100% |########################################################################|


In [9]:
def decision_tree(pos):

    x_train = train_data[pos][0]
    y_train = train_data[pos][1]

    classifier = DecisionTreeClassifier()
    classifier.fit(x_train, y_train)

    return classifier

predictors = {}  # dictionary to contain all decision trees given the position
#  key - position, value - decision tree for that position

for i in test_data.keys():
    if i > len(train_data) - 1:
        predictors[i] = decision_tree(len(train_data) - 1)

    else:
        predictors[i] = decision_tree(i)

In [11]:
pbar = ProgressBar()
print('[12/13]')

for i in pbar(
        log_test.keys()):  # adding an array with the encoding to the log_test dict. for every case in the test
    current = log_test[i][0]

    encoded = []  # list will contain all event names encoded into integers
    for g in current:
        encoded.append(int(le.transform([g])))
    encoded = np.array(encoded)
    log_test[i].append(encoded)

[12/13]


100% |########################################################################|


In [12]:
pbar = ProgressBar()
print('[13/13]')

for i in pbar(log_test.keys()):  # making predictions for every case in the log_test dict

    current_encoded = log_test[i][3]
    predictions = []  # list that will contain all predictions for a given case
    count = 0

    for x in current_encoded:

        # the if-else is a checks whether the case length is more than any case length observed in the train data
        if count >= len(train_data) - 1:  # if its in the train data we call the appropriate decision tree

            tree = predictors[len(train_data) - 1]  # calling the right tree given the position
            p = current_encoded[:(len(train_data))]  # taking the trace
            p = p.reshape(1, -1)
            pred = tree.predict(p)  # making a prediction
            pred_string = le.inverse_transform(pred)[0]  # transforming the prediction into a string
            predictions.append(pred_string)  # appending the prediction as a string to the log_test data



        else:  # if its not in the train data then we use the last observed decision tree from the train data

            tree = predictors[count]  # calling the right tree given the position
            p = current_encoded[:count + 1]  # taking the trace
            p = p.reshape(1, -1)  # we need to do that, idk why
            pred = tree.predict(p)  # making a prediction
            pred_string = le.inverse_transform(pred)[0]  # transforming the prediction into a string
            predictions.append(pred_string)  # appending the prediction as a string to the log_test data

        count += 1

    log_test[i].append(predictions)  # adding all predictions to the log_test of the current case

[13/13]


100% |########################################################################|


In [13]:
# making lists for every column we will have in the frame
case_names = []
event_names = []
timestamp = []
p_event = []
current_real = []

for i in log_test.keys():  # appending the right things to every list from the log_test file
    for x in range(len(log_test[i][0])):
        case_names.append(i)
        event_names.append(log_test[i][0][x])
        timestamp.append(log_test[i][1][x])
        p_event.append(log_test[i][4][x])
        current_real.append(log_test[i][2][x])

# dictionary that will be used to make the frame
frame_dict = {'Case_ID': case_names, 'Event_Name': event_names,
              'TimeStamp': timestamp, 'Next_Event': current_real, 'DTree_Event': p_event}

predicted_df = pd.DataFrame.from_dict(frame_dict)  # making a frame

In [14]:
predicted_df

,Case_ID,Event_Name,TimeStamp,Next_Event,DTree_Event
0,Application_1000334415,A_Create Application,2016-09-15 18:39:17.758,A_Submitted,A_Submitted
1,Application_1000334415,A_Submitted,2016-09-15 18:39:17.793,W_Handle leads,W_Handle leads
2,Application_1000334415,W_Handle leads,2016-09-15 18:39:17.950,W_Handle leads,W_Handle leads
3,Application_1000334415,W_Handle leads,2016-09-15 18:39:41.600,W_Complete application,W_Complete application
4,Application_1000334415,W_Complete application,2016-09-15 18:39:41.607,A_Concept,A_Concept
5,Application_1000334415,A_Concept,2016-09-15 18:39:41.610,W_Complete application,W_Complete application
6,Application_1000334415,W_Complete application,2016-09-16 12:16:13.728,W_Complete application,W_Complete application
7,Application_1000334415,W_Complete application,2016-09-16 12:18:25.724,W_Complete application,W_Complete application
8,Application_1000334415,W_Complete application,2016-09-16 13:23:45.017,A_Accepted,W_Complete application
9,Application_1000334415,A_Accepted,2016-09-16 13:31:41.123,O_Create Offer,O_Create Offer


In [15]:
#print('Len of DTree Algo Event Pred: {}'.format(len(predicted_df)))

# TODO: TIMESTAMPS REGRESSION

data_train['position_event'] = data_train.groupby('case concept:name').cumcount()
data_train['position_event'] = data_train['position_event'] + 1
data_train['week_day'] = data_train['event time:timestamp'].dt.dayofweek

# Encoding all event names into integers
cases = data_train['event concept:name'].unique().tolist()
cases.insert(0, 'New Case')
le_case = preprocessing.LabelEncoder()
le_case.fit(cases)

# Encoding lifecycle into integers
life = data_train['event lifecycle:transition'].unique().tolist()
le_life = preprocessing.LabelEncoder()
le_life.fit(life)

# Preprocess data for model train
# Event poistion
x_train_position = np.array(data_train['position_event']).reshape(-1, 1)[:]
# Previous event
x_train_prev = list(data_train['event concept:name'])
x_train_prev = le_case.transform(x_train_prev)
x_train_prev = np.array(x_train_prev).reshape(-1, 1)[:]
# Event
x_train_event = list(data_train['event concept:name'])
x_train_event.insert(len(data_train), 'New Case')
x_train_event = le_case.transform(x_train_event)
x_train_event = np.array(x_train_event).reshape(-1, 1)[1:]
# Day of the week previous event event
x_train_week = list(data_train['week_day'])
x_train_week = np.array(x_train_week).reshape(-1, 1)[:]
# Timestamp event
data_train[['event time:timestamp']] = data_train[['event time:timestamp']].astype(str)
x_train_date = list(data_train['event time:timestamp'])
x_train_date.insert(len(data_train), None)
x_train_date = np.array(x_train_date).reshape(-1, 1)[1:]
# Timestamp previous event
x_train_date_prev = list(data_train['event time:timestamp'])
x_train_date_prev = np.array(x_train_date_prev).reshape(-1, 1)[:]
# Event Lifecycle
x_train_life = list(data_train['event lifecycle:transition'])
x_train_life = le_life.transform(x_train_life)
x_train_life = np.array(x_train_life).reshape(-1, 1)[:]

# Length case for train set
cases = data_train.groupby(['case concept:name'])
per_case = pd.DataFrame({'no of events': cases['eventID '].count()})
lst_per_case = per_case["no of events"].tolist()
case_length = []
for length in lst_per_case:
    case_length.extend(repeat(length, length))
x_train_length_case = np.array(case_length).reshape(-1, 1)[:]

# Combine features for the model train
x_train_new = np.concatenate((x_train_position, x_train_prev, x_train_event, x_train_week, x_train_date,
                              x_train_date_prev, x_train_length_case, x_train_life), axis=1)

# Add features to new dataframe train
df_train = pd.DataFrame(data=x_train_new,
                        columns=['position_event', 'prev_event', 'event', 'week_day_prev', 'date', 'date_prev',
                                 'case_length', 'lifecycle'])
df_train.loc[df_train['position_event'] == df_train['case_length'], 'event'] = 5
df_train[['date', 'date_prev']] = df_train[['date', 'date_prev']].apply(pd.to_datetime)
df_train.loc[df_train['event'] == 5, 'date'] = None
df_train['in_between'] = (df_train['date'] - df_train['date_prev']).dt.days
df_train.loc[df_train['event'] == 5, 'in_between'] = 0
#df_train

# Train Dummies

# Implementing dummies train
df_train = pd.get_dummies(df_train, columns=['event', 'prev_event', 'week_day_prev', 'position_event', 'lifecycle'])
df_train = df_train.drop(['date', 'date_prev'], 1)
#df_train

# Test Data Preprocessing

# Add new useful columns for the model test
data_test['position_event'] = data_test.groupby('case concept:name').cumcount()
data_test['position_event'] = data_test['position_event'] + 1
data_test['week_day'] = data_test['event time:timestamp'].dt.dayofweek

predicted_events = predicted_df['DTree_Event'][:].tolist()
data_test['pred_event'] = predicted_events

# Preprocess data for model test
# Event poistion
x_test_position = np.array(data_test['position_event']).reshape(-1, 1)[:]
# Previous event
x_test_prev = data_test['event concept:name'].tolist()
x_test_prev = le_case.transform(x_test_prev)
x_test_prev = np.array(x_test_prev).reshape(-1, 1)[:]
# Predicted Event
x_test_event = data_test['pred_event'].tolist()
x_test_event = le_case.transform(x_test_event)
x_test_event = np.array(x_test_event).reshape(-1, 1)[:]
# Day of the week previous event
x_test_week = data_test['week_day'].tolist()
x_test_week = np.array(x_test_week).reshape(-1, 1)[:]
# Timestamp event
data_test[['event time:timestamp']] = data_test[['event time:timestamp']].astype(str)
x_test_date = list(data_test['event time:timestamp'])
x_test_date.insert(len(data_test), None)
x_test_date = np.array(x_test_date).reshape(-1, 1)[1:]
# Timestamp previous event
x_test_date_prev = list(data_test['event time:timestamp'])
x_test_date_prev = np.array(x_test_date_prev).reshape(-1, 1)[:]
# Event Lifecycle
x_test_life = data_test['event lifecycle:transition'].tolist()
x_test_life = le_life.transform(x_test_life)
x_test_life = np.array(x_test_life).reshape(-1, 1)[:]

# Length case for test set
test_cases = data_test.groupby(['case concept:name'])
per_case_test = pd.DataFrame({'no of events': test_cases['eventID '].count()})
lst_per_case_test = per_case_test["no of events"].tolist()
case_length_test = []
for length in lst_per_case_test:
    case_length_test.extend(repeat(length, length))
x_test_length_case = np.array(case_length_test).reshape(-1, 1)[:]

# Combine features for the model test
x_test_new = np.concatenate((x_test_position, x_test_prev, x_test_event, x_test_week, x_test_date, x_test_date_prev,
                             x_test_length_case, x_test_life), axis=1)

# Add features to new dataframe test
df_test = pd.DataFrame(data=x_test_new,
                       columns=['position_event', 'prev_event', 'event', 'week_day_prev', 'date', 'date_prev',
                                'case_length', 'lifecycle'])
df_test.loc[df_test['position_event'] == df_test['case_length'], 'date'] = None
df_test[['date', 'date_prev']] = df_test[['date', 'date_prev']].apply(pd.to_datetime)
df_test['in_between'] = (df_test['date'] - df_test['date_prev']).dt.days
df_test.loc[df_test['position_event'] == df_test['case_length'], 'in_between'] = 0
#df_test

# Remove cases with more events than the cases in the train set
df_test = df_test[df_test['case_length'] <= max(df_train['case_length'])]
#df_test

# Test Dummies

# Implementing dummies test
df_test = pd.get_dummies(df_test, columns=['event', 'prev_event', 'week_day_prev', 'position_event', 'lifecycle'])
df_test = df_test.drop(['date', 'date_prev'], 1)
#df_test

# Feature selection and model training


col_train = df_train.columns
col_test = df_test.columns
features = set(col_train).intersection(col_test)
features.discard('in_between')
X_train = df_train[features]  # Features
y_train = df_train['in_between']  # Target variable
X_test = df_test[features]  # Features
y_test = df_test['in_between']  # Target variable

# Training the algorithm
regressor = LinearRegression()
regressor.fit(X_train, y_train)


#print(regressor.intercept_)
#print(regressor.coef_)

# Evaluation


y_pred = regressor.predict(X_test)
#print('Len of DTree y_pred: {}'.format(len(y_pred)))

predicted_df['DTree_TimeDiff'] = y_pred

In [16]:
predicted_df

,Case_ID,Event_Name,TimeStamp,Next_Event,DTree_Event,DTree_TimeDiff
0,Application_1000334415,A_Create Application,2016-09-15 18:39:17.758,A_Submitted,A_Submitted,-1.914368
1,Application_1000334415,A_Submitted,2016-09-15 18:39:17.793,W_Handle leads,W_Handle leads,2.243713
2,Application_1000334415,W_Handle leads,2016-09-15 18:39:17.950,W_Handle leads,W_Handle leads,0.241089
3,Application_1000334415,W_Handle leads,2016-09-15 18:39:41.600,W_Complete application,W_Complete application,-0.567383
4,Application_1000334415,W_Complete application,2016-09-15 18:39:41.607,A_Concept,A_Concept,-0.196533
5,Application_1000334415,A_Concept,2016-09-15 18:39:41.610,W_Complete application,W_Complete application,12.240967
6,Application_1000334415,W_Complete application,2016-09-16 12:16:13.728,W_Complete application,W_Complete application,0.062317
7,Application_1000334415,W_Complete application,2016-09-16 12:18:25.724,W_Complete application,W_Complete application,7.304443
8,Application_1000334415,W_Complete application,2016-09-16 13:23:45.017,A_Accepted,W_Complete application,-1.594421
9,Application_1000334415,A_Accepted,2016-09-16 13:31:41.123,O_Create Offer,O_Create Offer,-1.320984
